In [1]:
import torch
import torch.nn as nn
from torch import optim

In [2]:
class net(nn.Module):
    
    def __init__(self):
        super(net, self).__init__()
        self.out = nn.Linear(10, 1)
        
    def forward(self, inp):
        return self.out(inp)

In [3]:
data = torch.randn(100, 10)
label = torch.randn(100,)
model = net()
optimizer = optim.SGD(model.parameters(), lr = 1e-3)
model_ = net()
model_.load_state_dict(model.state_dict())

In [4]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2316, -0.1861, -0.0710, -0.0396, -0.1752, -0.1690,  0.0250,  0.0418,
           0.1489,  0.1384]], requires_grad=True), Parameter containing:
 tensor([-0.0473], requires_grad=True)]

In [5]:
list(model_.parameters())

[Parameter containing:
 tensor([[-0.2316, -0.1861, -0.0710, -0.0396, -0.1752, -0.1690,  0.0250,  0.0418,
           0.1489,  0.1384]], requires_grad=True), Parameter containing:
 tensor([-0.0473], requires_grad=True)]

In [6]:
loss = (model(data) - label).pow(2).mean()
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [7]:
list(model_.parameters())

[Parameter containing:
 tensor([[-0.2316, -0.1861, -0.0710, -0.0396, -0.1752, -0.1690,  0.0250,  0.0418,
           0.1489,  0.1384]], requires_grad=True), Parameter containing:
 tensor([-0.0473], requires_grad=True)]

In [8]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2312, -0.1858, -0.0709, -0.0397, -0.1749, -0.1686,  0.0250,  0.0417,
           0.1486,  0.1383]], requires_grad=True), Parameter containing:
 tensor([-0.0474], requires_grad=True)]